#  자연어 처리를 위한 NLTK와 KoNLPy 설치하기

## NLTK와 NLTK Data 설치
- 엔엘티케이(NLTK)는 자연어 처리를 위한 파이썬 패키지

In [22]:
# !pip install nltk

In [23]:
import nltk
nltk.__version__

'3.6.2'

In [24]:
# 해당 코드를 실행 후에 NLTK 실습에 필요한 각종 패키지와 코퍼스를 다운로드할 수 있다
#nltk.download()

## KoNLPY 설치
- 코엔엘파이(KoNLPy)는 한국어 자연어 처리를 위한 형태소 분석기 패키지

In [25]:
#!pip install konlpy

In [26]:
import konlpy
konlpy.__version__

'0.5.2'

### 한국어 문장분류기(KSS : Korean Sentence Splitter) 설치

In [27]:
#! pip install kss

In [28]:
import kss

# 텍스트 전처리
- 토큰화 : 자연어 처리에서 크롤링 등으로 얻어낸 코퍼스 데이터가 필요에 맞게 전처리되지 않은 상태라면, 해당 데이터를 사용하고자하는 용도에 맞게 토큰화(tokenization) & 정제(cleaning) & 정규화(normalization)하는 일을 하게된다. 이렇듯 주어진 코퍼스에서 토큰이라 불리는 단위로 나눈 작업을 토큰화 작업이라고 부른다.

# 품사 태깅(POS-tagging)

- 위에서 영어 문장에 대해서 토큰화를 수행하고, 이어서 품사 태깅을 수행하였다. Penn Treebank POG Tags에서 PRP는 인칭 대명사, VBP는 동사, RB는 부사, VBG는 현재부사, IN은 전치사, NNP는 고유 명사, NNS는 복수형 명사, CC는 접속사, DT는 관사를 의미한다.

- 각각의 명칭에 대한 줄임말은 다음과 같고, 그 외에 참고할 말한 것들은 다음  사이트에서 확인할 수 있다. [참고링크](https://bluebreeze.co.kr/1357)
    - PRP : personal pronoun 인칭 대명사(I, you, he, she)
    - VBP : verb, non-third person singular present 3인칭이 아닌 현재형 동사
    - RB : adverb	부사
    - VBG : verb, gerund or present 동명사 또는 현재분사(~ing)
    - IN : perposition, subordinating conjunction 전치사 종속 접속사


## KoNLPY 활용한 한글 토큰화 실습
- 한국어 자연어 처리를 위해서 KoNLPy를 주로 쓴다. KoNLPy를 통해 사용할 수 있는 형태소 분석기로 Okt(Open Korea Text), 메캅(Mecab), 코모란(Komoran), 한나눔(Hannanum), 꼬꼬마(Kkma)가 있다.
- 한국어 NLP에서 형태소 분석기를 사용한다는 것은 단어 토큰화가 아니라 정확히는 형태소(morpheme) 단위로 형태소 토큰화(morpheme tokenization)를 수행하게 됨을 뜻한다.

# Word2Vec

**Word2Vec의 하이퍼파라미터 값**

- size = 워드 벡터의 특징 값. 즉, 임베딩 된 벡터의 차원.  
- window = 컨텍스트 윈도우 크기  
- min_count = 단어 최소 빈도 수 제한 (빈도가 적은 단어들은 학습하지 않는다.)  
- workers = 학습을 위한 프로세스 수  
- sg = 0은 CBOW, 1은 Skip-gram.  

## 한국어 데이터 다운로드 및 전처리

In [29]:
import urllib.request
from konlpy.tag import Okt
from gensim.models.word2vec import Word2Vec
import pandas as pd
import matplotlib.pyplot as plt

In [30]:
import os 

os.getcwd()

'/repo/course/sem21_01/youtube_summarizer/jupyter'

In [31]:
import json
import pandas as pd

path = '/repo/course/sem21_01/youtube_summarizer/dataset/article_dataset/train.jsonl'

### bfly train
sent_df = pd.read_json(path, lines=True, encoding="utf-8")
print('shape : {}'.format(sent_df.shape))
sent_df.head()

shape : (260697, 5)


,media,id,article_original,abstractive,extractive
0,부산일보,360972161,[지난해 고령화와 유례가 드문 겨울 한파 등 영향으로 우리나라 사망자 수가 통계 작...,통계청이 발표한 '2018년 사망원인통계'를 보면 지난해 총 사망자 수는 관련 통계...,"[4, 11, 18]"
1,중도일보,356659913,[서산시의회(의장 임재관) 가충순·이수의 의원이 (사)한국지역신문협회에서 수여하는 ...,서산시 가충순 의원과 이수의 의원이 활발한 의정활동을 펼친 감사의 표시로 한국지역신...,"[1, 3, 4]"
2,무등일보,351718460,[지난 2004년 시작해 조선대 학생들의 대표적인 행사로 자리매김한 ‘조선대 국토대...,‘조선대의 새로운 비상을 꿈꾸다’를 슬로건으로 진행되어 단체생활을 통해 협동심과 ...,"[0, 2, 4]"
3,이데일리,335868123,[서울시는 신학기가 시작되는 다음달 4일부터 고등학교 3학년 무상급식을 실시한다고 ...,서울시가 다음달 4일부터 서울 시내 319개 고등학교 3학년 8만4700명을 대상으...,"[0, 1, 2]"
4,서울신문,351443347,[미국인 선교사가 우간다에서 의사 행세를 하며 의료 시설을 운영한 혐의로 지역 시민...,미국인 선교사가 우간다에서 의사 행세를 하며 두 아이의 죽음과 관련돼 있다며 지역 ...,"[0, 1, 2]"


In [32]:
texts = ' '.join([article[0] for article in sent_df['article_original'].values])

In [33]:
import re
# 영어 단어가 생각보다 많네?
eng = re.findall('([a-zA-Z]{1,})', texts)
len(eng)

109150

In [ ]:
# mecab - 방법1
# https://sosomemo.tistory.com/30
# Mecab설치
'''bash
cd /tmp
sudo wget https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
sudo tar xvf mecab-0.996-ko-0.9.2.tar.gz

cd /tmp/mecab-0.996-ko-0.9.2
sudo ./configure
sudo make check
sudo make install
'''

'''bash
cd /tmp
wget https://bitbucket.org/eunjeon/mecab-ko-dic/downloads/mecab-ko-dic-2.1.1-20180720.tar.gz
tar zxvf mecab-ko-dic-2.1.1-20180720.tar.gz

cd /tmp/mecab-ko-dic-2.1.1-20180720
sudo ./autogen.sh
sudo ./configure
sudo make
sudo make install
'''

'''bash
cd /tmp
git clone https://bitbucket.org/eunjeon/mecab-python-0.996.git
cd mecab-python-0.996
python3 setup.py build
python3 setup.py install
'''

'bash\ncd /tmp\ngit clone https://bitbucket.org/eunjeon/mecab-python-0.996.git\ncd mecab-python-0.996\npython3 setup.py build\npython3 setup.py install\n'

In [ ]:
# mecab 설치 - 방법1
# ! pip install konlpy
# ! sudo apt-get install curl git
# ! bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

Reading package lists... Done
Building dependency tree       
Reading state information... Done
curl is already the newest version (7.58.0-2ubuntu3.13).
git is already the newest version (1:2.17.1-1ubuntu0.8).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 55 not upgraded.
mecab-ko is already installed
mecab-ko-dic is already installed
mecab-python is already installed
Done.


In [34]:
from konlpy.tag import Okt  
from konlpy.tag import Mecab
mecab = Mecab()
#okt = Okt()

In [35]:
#stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','을','를','으로','자','에','와','한','하다', '합니다', '입니다']
stopwords = ['을', '를', '이', '가', '은', '는', '의', '에', '와', '으로', '합니다', '입니다']

In [36]:
import re

In [37]:
tokenized_news_data = []
for news in sent_df['article_original']:
    for sentence in news:
        #print(f'previous : {sentence}')
        tmp_sen = re.sub('[^가-힣a-z]', ' ', sentence) # 영어 소문자와 한글을 제외한 모든 문자를 제거
        temp_X = mecab.morphs(tmp_sen) # 토큰화
        temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
        #print(f'after : {temp_X}')
        tokenized_news_data.append(temp_X)

In [38]:
import pickle

pk_path = '/repo/course/sem21_01/youtube_summarizer/dataset/article_tokenized/tokenized_news_data.pkl'

# save pickle file
with open(pk_path, 'wb') as f:
    pickle.dump(tokenized_news_data, f)

In [20]:
import pickle

pk_path = '/repo/course/sem21_01/youtube_summarizer/dataset/article_tokenized/tokenized_news_data.pkl'

# load pickle file
with open(pk_path, 'rb') as f:
     tokenized_news_data = pickle.load(f)

In [39]:
tokenized_news_data[0]

['지난해',
 '고령화',
 '유례',
 '드문',
 '겨울',
 '한파',
 '등',
 '영향',
 '우리나라',
 '사망자',
 '수',
 '통계',
 '작성',
 '이후',
 '가장',
 '많',
 '았',
 '다']

In [40]:
sent_df['article_original'][0][0]

'지난해 고령화와 유례가 드문 겨울 한파 등 영향으로 우리나라 사망자 수가 통계 작성 이후 가장 많았다.'

In [41]:
tokenized_news_data[3]

['작년', '사망자', '만', '명', '역대', '최다']

In [42]:
sent_df['article_original'][0]

['지난해 고령화와 유례가 드문 겨울 한파 등 영향으로 우리나라 사망자 수가 통계 작성 이후 가장 많았다.',
 '폐렴과 치매의 일종인 알츠하이머병은 지난해 사망원인 순위 3위와 9위로 전년보다 각각 한 단계, 두 단계 상승하는 등 노인성 질병에 의한 사망률이 급증하는 추세다.',
 '‘연령표준화 사망률’(표준인구 10만 명당 사망자 수)은 울산·충북·부산 순으로 높게 나타났다.',
 '■작년 사망자 29만 8820명, 역대 최다',
 "24일 통계청이 발표한 '2018년 사망원인통계'를 보면 지난해 총 사망자 수는 전년 대비 4.7%(1만 3286명) 증가한 29만 8820명으로 관련 통계를 작성한 1983년 이후 가장 많았으며, 5년 연속 증가세를 보였다.",
 '통계청은 인구 구조의 고령화와 지난해 1~2월 유례가 드문 한파 등을 그 원인으로 꼽았다.',
 '지난해 조사망률(인구 10만 명당 사망자 수) 역시 582.5명으로 전년보다 4.5%(25.1명) 증가해 5년 연속 늘었다.',
 '특히 80세 이상의 사망자가 전체 사망자의 절반에 가까운 46.3%로, 10년 전보다 14.3%포인트(P)나 증가했다.',
 '■폐렴·알츠하이머병 사망률 순위 ‘껑충’',
 '사망원인별로 보면 지난해 암(악성신생물)에 의한 사망률(이하 인구 10만 명당 사망자 수)은 154.3명으로 전년보다 0.2% 증가했다.',
 '1983년 관련 통계를 집계한 이래 줄곧 암이 사망원인 1위로 집계됐다.',
 '특히 폐렴(4위→3위)과 치매의 일종인 알츠하이머병(11위→9위)에 의한 사망률 순위 상승이 두드러졌다.',
 '폐렴 사망률은 2004년 10위에서 꾸준히 순위가 상승하고 있고, 알츠하이머병 사망률 역시 통계 작성 이래 10대 사인에 처음 포함됐다.',
 '지난해 알츠하이머병에 의한 사망률은 12.0명으로 전년(9.8명) 대비 22.5% 증가했다.',
 '알츠하이머병 사망률은 10년 전(3.8명)과 비교하면 무려 214.2% 증가했다.',
 '폐렴 사망률은 45.4명으로 전년(

### import pretrained model
**Pretraining 환경**
~~~
def train_word2vec(corpus_fname, model_fname):
    make_save_path(model_fname)
    corpus = Word2VecCorpus(corpus_fname)
    model = Word2Vec(corpus, size=100, workers=4, sg=1)
    model.save(model_fname)
~~~

In [43]:
from gensim.models import Word2Vec

In [44]:
import gensim
from gensim.models import Word2Vec

w2v_path = '/repo/course/sem21_01/youtube_summarizer/dataset/pretrained_word2vec/word2vec'

model = Word2Vec.load(w2v_path)

model


In [45]:
model.wv.save_word2vec_format("/repo/course/sem21_01/youtube_summarizer/dataset/pretrained_word2vec/word2vec.bin.gz", binary=False)

### finetune pretrained model

In [46]:
# define new model
model_2 = Word2Vec(size=100, workers=4, sg=1)

# make voacb
model_2.build_vocab(tokenized_news_data)

In [47]:
total_examples = model_2.corpus_count
print(total_examples)

3300370


In [48]:
model_2.build_vocab([list(model.wv.vocab.keys())], update=True)
model_2.intersect_word2vec_format("/repo/course/sem21_01/youtube_summarizer/dataset/pretrained_word2vec/word2vec.bin.gz", binary=False)


In [49]:
model_2.train(tokenized_news_data, total_examples=total_examples, epochs=model_2.iter)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  """Entry point for launching an IPython kernel.


(378023619, 437498110)

In [50]:
# 어떤 단어들이 늘어났는가 확인

for k in model_2.wv.vocab:
    result = model.wv.vocab.get(k, "0")
    if result=="0":
        print(k, end=",")

소요액,채태병,음식창,박병술,여년,김병숙,순매도,ublic,쑥스럽,쑥스러운,이승동,담보금,박남천,아침상,uman,esource,정봉기,생착률,정만기,창호재,농막,inance,ower,oom,흥부가,동리국,치러낼,탁상행정,이창형,반측,rivate,wards,박창완,도석구,양원돈,냉열,물어봐야,민경서,김용장,uard,eace,정대진,고규창,ngineering,uclear,nergy,treemm,imaeil,ake,illennial,rands,arget,투찰,면키,molit,jindo,본품,속충,긴들,ody,rame,olorado,eep,백종빈,pring,gor,높여왔,철음식,산화바나듐,이병래,ongress,면역원성,삼척항,심재권,월동채소,떠맡긴,최윤규,ictory,aik,liday,peech,신동렬,rogram,곽안나,정개특위,전지분유,cean,tore,arina,치금,하현회,kozmo,시집와서,수검자,arket,정회진,약건,주택건설공사,떨어진데다,전화료,끌려다닌,서상영,릭스미스,김삼호,조영임,고서곤,문종배,노환중,앞장선다는,immy,허인철,박세열,gcon,ights,illage,ueen,iamond,순이자,이덕선,egulation,andbox,icense,sset,디스인플레이션,저물가,개인서비스,거부지,ove,발령일,건으로,backstop,ilm,estival,주민자,낮춰줄,egno,서영경,이득수,약보합세,굴토,ommercial,peration,여객실,김병내,장생포항,황인홍,yoonki,강신덕,가을갈이,경영비,반등세,감척,이은학,민세희,황성태,임숙자,대한볼링협회,양껏,mployment,utlook,uture,eeling,omething,김황배,강보합세,이르면,고광현,전이암,대한체조협회,ulture,ourism,avel,iesta,이숙애,곽병창,넓혀갈,최해영,박일평,드영,이돈흥,송필용,정상섭,ive,hanks,유영상,orporate,송영관,bhc,조인동,조현배,다산역,이원율,김봉운,oilet,lobal,arketplace,ipeline,evice,erver,심축,

In [51]:
model_2.save("/repo/course/sem21_01/youtube_summarizer/dataset/pretrained_word2vec/word2vec_finetuned.bin")

In [57]:
finetuned_wv = model_2.wv

In [52]:
# https://www.infoking.site/16
# https://jjdeeplearning.tistory.com/6

### Load Fine-tuned Word2Vec Model & Vectors

In [53]:
import gensim
from gensim.models import Word2Vec


model_prac = Word2Vec.load("/repo/course/sem21_01/youtube_summarizer/dataset/pretrained_word2vec/word2vec_finetuned.bin")

model_prac

In [58]:
from gensim.models import KeyedVectors

finetuned_wv.save("/repo/course/sem21_01/youtube_summarizer/dataset/pretrained_word2vec/finetuned_wv.kv")

In [61]:
reloaded_word_vectors = KeyedVectors.load("/repo/course/sem21_01/youtube_summarizer/dataset/pretrained_word2vec/finetuned_wv.kv")
reloaded_word_vectors

In [65]:
reloaded_word_vectors["안녕"]

array([ 0.2991806 ,  0.22248697,  0.43511766,  0.25895345, -0.06910251,
        0.17864834,  0.5707439 ,  0.33198267, -0.56168944,  0.15012914,
       -0.48048452,  0.10886598,  0.95291424, -0.6656376 , -0.01721039,
       -0.12382434,  0.33175135, -0.1997768 ,  0.45495436,  0.05084743,
        1.1117018 ,  0.07934755, -0.09810785, -0.20020787, -0.09609537,
        0.08259846,  0.28931755, -0.35209242, -0.06767287, -0.14012145,
       -0.21060923,  0.14916308, -0.11388788, -0.07301804,  0.28009343,
       -0.07612846, -0.15479222,  0.00236909, -0.42169824, -0.05550557,
        0.05788445, -0.01412638,  0.73523134, -0.16871905, -0.11922143,
       -0.16073903,  0.25312254, -0.4280252 , -0.14929976, -0.33271357,
        0.47214702,  0.01922962,  0.12974948, -0.649931  ,  0.07555343,
       -0.0176564 , -0.18954116, -0.6716446 , -0.07588163,  0.03062781,
       -0.05300049, -0.3634111 ,  0.10411528,  0.9098448 ,  0.12853889,
        0.53329414,  0.12131036,  0.01449181,  0.27068338, -0.41

In [64]:
reloaded_word_vectors.get_vector("안녕")

array([ 0.2991806 ,  0.22248697,  0.43511766,  0.25895345, -0.06910251,
        0.17864834,  0.5707439 ,  0.33198267, -0.56168944,  0.15012914,
       -0.48048452,  0.10886598,  0.95291424, -0.6656376 , -0.01721039,
       -0.12382434,  0.33175135, -0.1997768 ,  0.45495436,  0.05084743,
        1.1117018 ,  0.07934755, -0.09810785, -0.20020787, -0.09609537,
        0.08259846,  0.28931755, -0.35209242, -0.06767287, -0.14012145,
       -0.21060923,  0.14916308, -0.11388788, -0.07301804,  0.28009343,
       -0.07612846, -0.15479222,  0.00236909, -0.42169824, -0.05550557,
        0.05788445, -0.01412638,  0.73523134, -0.16871905, -0.11922143,
       -0.16073903,  0.25312254, -0.4280252 , -0.14929976, -0.33271357,
        0.47214702,  0.01922962,  0.12974948, -0.649931  ,  0.07555343,
       -0.0176564 , -0.18954116, -0.6716446 , -0.07588163,  0.03062781,
       -0.05300049, -0.3634111 ,  0.10411528,  0.9098448 ,  0.12853889,
        0.53329414,  0.12131036,  0.01449181,  0.27068338, -0.41

### Getting Sentence E,e